# Connect Four 
by 4조 2015140521 김나연, 2015140525 이수민


In [1]:
# connect4 게임 형식 코드 출처 : https://github.com/KeithGalli/Connect4-Python/blob/master/connect4_with_ai.py
# 참조한 부분 : 게임 진행되는 전체적인 흐름, 원래있었던 minimax 함수 기본 구조 / 차이점 : 휴리스틱함수 (자세한건 중간중간 서술)

import numpy as np
import random
import pygame
import sys
import math

BLUE = (0,0,255)
WHITE = (255,255,255)
RED = (255,0,0)
BLACK = (0,0,0)
P = (180,180,255)
R = (255,70,70)
SKYB = (135,245,245)



ROW_COUNT = 6
COLUMN_COUNT = 7
free_cells = 42
PLAYER = 0
AI = 1

EMPTY = 0
PLAYER_PIECE = 1
AI_PIECE = 2

#threat를 위한 전역변수
ODD = -1
EVEN = -1

WINDOW_LENGTH = 4

i = 0

def create_board():
    board = np.zeros((ROW_COUNT,COLUMN_COUNT))
    return board

def drop_piece(board, row, col, piece):
    board[row][col] = piece

def is_valid_location(board, col):
    return board[ROW_COUNT-1][col] == 0

def get_next_open_row(board, col):
    for r in range(ROW_COUNT):
        if board[r][col] == 0:
            return r

def print_board(board):
    print(np.flip(board, 0))

def winning_move(board, piece):
    # Check horizontal locations for win
    for c in range(COLUMN_COUNT-3):
        for r in range(ROW_COUNT):
            if board[r][c] == piece and board[r][c+1] == piece and board[r][c+2] == piece and board[r][c+3] == piece:
                return True

    # Check vertical locations for win
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT-3):
            if board[r][c] == piece and board[r+1][c] == piece and board[r+2][c] == piece and board[r+3][c] == piece:
                return True

    # Check positively sloped diaganols
    for c in range(COLUMN_COUNT-3):
        for r in range(ROW_COUNT-3):
            if board[r][c] == piece and board[r+1][c+1] == piece and board[r+2][c+2] == piece and board[r+3][c+3] == piece:
                return True

    # Check negatively sloped diaganols
    for c in range(COLUMN_COUNT-3):
        for r in range(3, ROW_COUNT):
            if board[r][c] == piece and board[r-1][c+1] == piece and board[r-2][c+2] == piece and board[r-3][c+3] == piece:
                return True

def is_terminal_node(board):
    return winning_move(board, PLAYER_PIECE) or winning_move(board, AI_PIECE) or len(get_valid_locations(board)) == 0

def get_valid_locations(board):
    valid_locations = []
    for col in range(COLUMN_COUNT):
        if is_valid_location(board, col):
            valid_locations.append(col)
    return valid_locations

def draw_board(board):
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT):
            pygame.draw.rect(screen, SKYB, (c*SQUARESIZE, r*SQUARESIZE+SQUARESIZE, SQUARESIZE, SQUARESIZE))
            pygame.draw.circle(screen, WHITE, (int(c*SQUARESIZE+SQUARESIZE/2), int(r*SQUARESIZE+SQUARESIZE+SQUARESIZE/2)), RADIUS)
    
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT):        
            if board[r][c] == PLAYER_PIECE:
                pygame.draw.circle(screen, R, (int(c*SQUARESIZE+SQUARESIZE/2), height-int(r*SQUARESIZE+SQUARESIZE/2)), RADIUS)
            elif board[r][c] == AI_PIECE: 
                pygame.draw.circle(screen, P, (int(c*SQUARESIZE+SQUARESIZE/2), height-int(r*SQUARESIZE+SQUARESIZE/2)), RADIUS)
    pygame.display.update()

pygame 1.9.5
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def score_position(board, piece):
    score = 0
    
    aithreat, plthreat, aidoublethreat, pldoublethreat, oddevenmixed, temp_score = evaluation(board, piece-1)
    
    score += temp_score

    return score, aithreat, plthreat, aidoublethreat, pldoublethreat, oddevenmixed

#minimax 알고리즘 코드 - 김나연 (기본구조는 위에서 언급한 https://github.com/KeithGalli/Connect4-Python/blob/master/connect4_with_ai.py에 있는것을 활용)
#첫 수를 둘 때 valid_locations에서 3을 빼서 3에 못두게 함
#print를 하기 위해서 aithreat, plthreat, aidoublethreat, pldoublethreat, oddevenmixed 값들을 넘겨주었다.
#AI부분에서만 best랑 secondbest를 업데이트 해주었다.
def minimax(board, depth, alpha, beta, maximizingPlayer, line_col, firstcol): #return : col , score, aithreat, plthreat, aidoublethreat, pldoublethreat, oddevenmixed 
    global i
    valid_locations = get_valid_locations(board)
    #3을 못두게 하는 부분
    if free_cells == 42 and depth == 6 and 3 in valid_locations:
        valid_locations.remove(3)
    random.shuffle(valid_locations)
    is_terminal = is_terminal_node(board)
    if depth == 0 or is_terminal:
        if is_terminal:
            if winning_move(board, AI_PIECE):
                temp = [None,100000000000000,100000000000000,100000000000000,100000000000000,100000000000000,100000000000000,100000000000000]
                return (temp, temp)
            elif winning_move(board, PLAYER_PIECE):
                temp = [None,-100000000000000,-100000000000000,-100000000000000,-100000000000000,-100000000000000,-100000000000000,-100000000000000]
                return (temp, temp)
            else: # Game is over, no more valid moves
                temp = [None, 0,0,0,0,0,0,0]
                return (temp, temp)
        else: # Depth is zero
            score, aithreat, plthreat, aidoublethreat, pldoublethreat, oddevenmixed = score_position(board, AI_PIECE)
            line = line_col[firstcol]
            return ([None, score + line,aithreat, plthreat, aidoublethreat, pldoublethreat, oddevenmixed, line],[None, 0,0,0,0,0,0,0])
    #column, value, best, secondbest,aithreat, plthreat, aidoublethreat, pldoublethreat, oddevenmixed
    if maximizingPlayer:
        column = random.choice(valid_locations)
        value = [column,-math.inf,0,0,0,0,0,0,0,0]
        second_value = [column,-math.inf,0,0,0,0,0,0,0,0]
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = board.copy()
            drop_piece(b_copy, row, col, AI_PIECE)
            if depth == 6 :
                firstcol = col #column이 차례대로 되니까 유효하다 첫번째수를 1로 뒀을때 쭉 가고 그다음 2로 뒀을때 쭉
            #i += 1
            #print("max : ", depth, i, "first :",firstcol,"now : ", col)
            new_value = minimax(b_copy, depth-1, alpha, beta, False, line_col, firstcol)[0][1:8]           
            
            
            if new_value[0] > value[1] :
                #print("BEST!!!!!!!!!!!!!!!!!!!!!",firstcol, col, new_score, value)
                second_value = value
                new_value.insert(0,col)
                value = new_value
                
                
            alpha = max(alpha, value[1])
            if alpha >= beta:
                break
        return value, second_value

    else: # Minimizing player
        
        column = random.choice(valid_locations)
        value = [column,math.inf,0,0,0,0,0,0,0,0]
        second_value = [column,math.inf,0,0,0,0,0,0,0,0]
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = board.copy()
            drop_piece(b_copy, row, col, PLAYER_PIECE)

            new_value = minimax(b_copy, depth-1, alpha, beta, True, line_col,firstcol)[0][1:8]            

            if new_value[0] < value[1] :
                second_value = value
                new_value.insert(0,col)
                value = new_value
                
            beta = min(beta, value[1])
            if alpha >= beta:
                break
                
        return value, second_value

#한 수를 두었을때 게임이 끝나는 수가 있는지 확인하는 함수 - 김나연
def check_three(board, piece) :    
    
    row = -1
    col = -1
    valid_loc = []
    
    #놓을 수 있는 위치 구하기
    for c in range(COLUMN_COUNT) :
        if is_valid_location(board, c) :
            valid_loc.append([get_next_open_row(board,c), c])
     
    #두었을때 연속하는가?    
    for x in range(len(valid_loc)) :
        b_copy = board.copy()
        drop_piece(b_copy, valid_loc[x][0], valid_loc[x][1]  , piece)
        if winning_move(b_copy, piece) :
            return valid_loc[x][0], valid_loc[x][1]                
    
    #없다면 -1 return
    return row, col

#threat를 위한 부분

#총 heuristic 함수 값을 구하는 함수 - 김나연
def evaluation(board, turn) :
    score = 0
    
    #threat값계산
    AIodd, AIeven, PLodd, PLeven, AIdoublethreat, PLdoublethreat = getthreat(board)
    oddevenmixed = 0
    
    #일단 threat 개수 만큼 점수 가중치
    if turn == AI :
        oddevenmixed = ODDEVENMIXED (AI, AIodd, PLodd, PLeven) *200
        score = score + len(AIodd)*100 + len(AIeven)*100 - len(PLodd)*100 - len(PLeven)*100 + oddevenmixed - len(PLdoublethreat)*50  #+ len(AIdoublethreat)*100
    else :
        oddevenmixed = ODDEVENMIXED (PLAYER, PLodd, AIodd, AIeven)*100
        score = score - len(AIodd)*100 - len(AIeven)*100 + len(PLodd)*100 + len(PLeven)*100 + oddevenmixed - len(AIdoublethreat)*50 #+ len(PLdoublethreat)*100
    
    aithreat = len(AIodd) + len(AIeven)
    plthreat = len(PLodd) + len(PLeven)
    aidoublethreat = len(AIdoublethreat)
    pldoublethreat = len(PLdoublethreat)
    
    return aithreat, plthreat, aidoublethreat, pldoublethreat, oddevenmixed, score

#threat값 계산하는 함수 - 김나연
#threat란 수를 두었을때 4개가 연속하게 되는 위치를 가지는 경우. 한 column에 여러개가 있으면 가장 아래의 것으로 계산한다.
#아이디어 차용은 Allis, V. (1988). A knowledge-based approach of Connect-Four-the game is solved: White wins. 문서 참고. 코드는 직접 짰음
def getthreat(board): 
    
    AIodd = []
    AIeven = []
    PLodd = []
    PLeven = []
    AIdoublethreat = []
    PLdoublethreat = []
    
    #AI
    for y in range(COLUMN_COUNT) :
        for x in range(ROW_COUNT-1, -1, -1) :
            if board[x][y] == 0 :                
                b_copy = board.copy()
                drop_piece(b_copy, x, y, AI_PIECE)
                if winning_move(b_copy, AI_PIECE) :
                    if x%2 == 1 : #------------------------------------------지금은 0부터 시작하니까
                        AIeven.append([x,y])
                    else :
                        AIodd.append([x,y])
                    break
    
    #PLAYER
    for y in range(COLUMN_COUNT) :
        for x in range(ROW_COUNT-1, -1, -1) :
            if board[x][y] == 0 :                
                b_copy = board.copy()
                drop_piece(b_copy, x, y, PLAYER_PIECE)
                if winning_move(b_copy, PLAYER_PIECE) :
                    if x%2 == 1 :
                        PLeven.append([x,y])
                    else :
                        PLodd.append([x,y])
                    break
                    
    for x in range(ROW_COUNT) :
        for y in range(COLUMN_COUNT) :
            if board[x][y] == 0 :
                if getdoublethreat(x,y,board,AI_PIECE) :
                    AIdoublethreat.append([x,y])
                if getdoublethreat(x,y,board, PLAYER_PIECE) :
                    PLdoublethreat.append([x,y])
                
    return AIodd, AIeven, PLodd, PLeven, AIdoublethreat, PLdoublethreat     

#doublethreat값 계산하는 함수 - 김나연
#doublethreat란 수를 두었을때, 4개가 연속할수있는 경우가 2개가 되게 해주는 수
#아이디어 차용은 Allis, V. (1988). A knowledge-based approach of Connect-Four-the game is solved: White wins. 문서 참고. 코드는 직접 짰음
def getdoublethreat(x,y, board, piece) :
    
    if x == 1 : #오른쪽만 검사
        if y == 1 : #밑으로만 검사
            if board[x+1][y] == piece and board[x+2][y] == piece and board[x+1][y+1] == piece and board[x+2][y+2] == piece :
                return True
            
        elif y == 4 : #위로만 검사
            if board[x+1][y] == piece and board[x+2][y] == piece and board[x+1][y-1] == piece and board[x+2][y-2] == piece :
                return True
        
        elif y == 2 or y == 3 :
            if (board[x+1][y] == piece and board[x+2][y] == piece and board[x+1][y+1] == piece and board[x+2][y+2] == piece) or \
            (board[x+1][y] == piece and board[x+2][y] == piece and board[x+1][y+1] == piece and board[x+2][y+2] == piece) :
                return True
            
    
    elif x == 5 : #왼쪽만 검사
        if y == 1 : #밑으로만 검사
            if board[x-1][y] == piece and board[x-2][y] == piece and board[x-1][y+1] == piece and board[x-2][y+2] == piece :
                return True
                        
        elif y == 4 : #위로만 검사
            if board[x-1][y] == piece and board[x-2][y] == piece and board[x-1][y-1] == piece and board[x-2][y-2] == piece :
                return True
        
        elif y == 2 or y == 3 :
            if (board[x-1][y] == piece and board[x-2][y] == piece and board[x-1][y+1] == piece and board[x-2][y+2] == piece) or \
            (board[x-1][y] == piece and board[x-2][y] == piece and board[x-1][y-1] == piece and board[x-2][y-2] == piece) :
                return True
        
    elif x == 2 or x == 3 or x == 4 :
        if y == 1 : #밑으로만 검사
            if (board[x+1][y] == piece and board[x+2][y] == piece and board[x+1][y+1] == piece and board[x+2][y+2] == piece) or \
            (board[x-1][y] == piece and board[x-2][y] == piece and board[x-1][y+1] == piece and board[x-2][y+2] == piece) :
                return True
            
        elif y == 4 : #위로만 검사
            if (board[x+1][y] == piece and board[x+2][y] == piece and board[x+1][y-1] == piece and board[x+2][y-2] == piece) or \
            (board[x-1][y] == piece and board[x-2][y] == piece and board[x-1][y-1] == piece and board[x-2][y-2] == piece) :
                return True
        
        elif y == 2 or y == 3 :
            if (board[x+1][y] == piece and board[x+2][y] == piece and board[x+1][y+1] == piece and board[x+2][y+2] == piece) or \
            (board[x+1][y] == piece and board[x+2][y] == piece and board[x+1][y+1] == piece and board[x+2][y+2] == piece) or \
            (board[x-1][y] == piece and board[x-2][y] == piece and board[x-1][y+1] == piece and board[x-2][y+2] == piece) or \
            (board[x-1][y] == piece and board[x-2][y] == piece and board[x-1][y-1] == piece and board[x-2][y-2] == piece) :
                return True
            
    return False

#휴리스틱 함수 - 이수민
# 출처 : https://www.gamedev.net/forums/topic/225611-connect-4-evaluation/ 아이디어 차용 여기부터 oddevenmixed까지

#-----------------------------------------------------------------수민
##ODD
def ODD_NUM(First_odd, Second_odd):
    return len(Second_odd) - len(First_odd)

##EVEN
def EVEN_NUM(Second_even):
    return len(Second_even)

##MIXED
def MIXED_NUM(First_odd, Second_odd):
    sum_odd = First_odd + Second_odd
    mixed_col=[]
    for x in range(len(First_odd)):
        for y in range(len(Second_odd)):
            if First_odd[x][0]==Second_odd[y][0]:
                i = First_odd[x][0]
                mixed_col.append(i)
    return  len(set(mixed_col))



def ODDEVENMIXED (WHO, First_odd, Second_odd, Second_even): ##처음 놓는 사람의 기준으로 WIN 값 나옴
    Odd = ODD_NUM(First_odd, Second_odd)
    Mixed = MIXED_NUM(First_odd, Second_odd)
    Even = EVEN_NUM(Second_even)
    WIN = 0 ##WIN = 1 이면 유리한 상황, WIN=0 이면 비기는 상황, WIN = -1이면 지는 상황
    if (Odd<0):
         WIN = 1
    if (Odd == 0):
        if (Mixed%2 == 1): ##mixed is odd
            WIN = 1
        if(Mixed%2 == 0):##mixed is even
            if(Mixed ==0):
                if(Even ==0):
                    WIN = 0
                if(Even >0):
                     WIN = -1
            if(Mixed>0):
                      WIN = -1
    if (Odd == 1):
        if (Mixed%2 == 1): ##mixed is odd
             WIN = -1
        if (Mixed%2 == 0):##mixed is even
             WIN = -1
    if(Odd>1):
         WIN = -1
    if WHO == EVEN:
        WIN = -WIN
    return WIN

#-----------------------------------4칸line 가능 여부 함수 시작--------------------------
def check_avail_line(window, piece): #check_window 변경
    avail_line= 0
    opp_piece = PLAYER_PIECE
    if piece == PLAYER_PIECE:
        opp_piece = AI_PIECE
    
    #if window.count(piece)+window.count(EMPTY) == 4:
       # avail_line = 1
        
    if window.count(piece) == 0 and window.count(EMPTY) == 4 :
        avail_line += 1
    elif window.count(piece) == 1 and window.count(EMPTY) == 3 :
        avail_line += 3
    elif window.count(piece) == 2 and window.count(EMPTY) == 2 :
        avail_line += 4
    elif window.count(opp_piece) >= 1  :
        avail_line -= 1



     
    return avail_line


def line_evaluation(board, piece, row, col): #score_position(board, piece) 변경
    avail_line_num = 0
    
    ## Horizontal check
    start = max(col-3,0)
    end = min(col+3, 6)
    for r in range(end - start - 2) :
        window = [board[row][start + r + i] for i in range(WINDOW_LENGTH)]
        avail_line_num += check_avail_line(window, piece)        
 
    ##Vertical check
    bottom = max(row-3,0)
    top = min(row+3,5)
    for r in range(top - bottom - 2) :
        window = [board[bottom + i + r][col] for i in range(WINDOW_LENGTH)]
        avail_line_num += check_avail_line(window, piece)

    ##diagonal check 
    #//
    left = min(col-start, row-bottom)
    right = min(end-col, top-row)
    start_x = col - left
    start_y = row - left
    end_x = col + right
    end_y = row + right
    
    #print("why",col, row, right, left, right + left - 2 )
    for r in range(right + left - 2) :
        window = [board[start_y+i+r][start_x+i+r] for i in range(WINDOW_LENGTH)]
        avail_line_num += check_avail_line(window, piece)
       
    left = min(col - start, top-row)
    right = min(end - col, row-bottom)
    start_x = col - left
    start_y = row + left
    end_x = col + right
    end_y = row - left
    #\\        
    for r in range(right + left - 2) :
        window = [board[start_y - r - i][start_x + r+i] for i in range(WINDOW_LENGTH)]
        avail_line_num += check_avail_line(window, piece)
        #print('왼쪽대각선', r, avail_line_num)
                    
    return avail_line_num

#-----------------------------------4칸line 가능 여부 함수끝--------------------------

In [3]:
board = create_board()
print_board(board)
game_over = False

pygame.init()

SQUARESIZE = 100

width = COLUMN_COUNT * SQUARESIZE
height = (ROW_COUNT+1) * SQUARESIZE

size = (width, height)

RADIUS = int(SQUARESIZE/2 - 5)

screen = pygame.display.set_mode(size)
draw_board(board)
pygame.display.update()

#----------------
myfont = pygame.font.SysFont("comicsansms",35)
notification = pygame.font.SysFont("monospace",30)

#턴 정하기 + 턴 저장(threat)
pygame.draw.rect(screen, WHITE, (0,0, width, SQUARESIZE))
label = myfont.render("press 0 : you first / 1 : AI first", 1, BLACK)
screen.blit(label, (40,10))
pygame.display.update()

inputinput = True
while  inputinput:     
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_0 or event.key == pygame.K_KP0:
                turn = PLAYER
                ODD = PLAYER
                EVEN = AI
                inputinput = False
            elif event.key == pygame.K_1 or event.key == pygame.K_KP1:
                turn = AI
                ODD = AI
                EVEN = PLAYER
                inputinput = False
                
label = myfont.render(" ", 1, BLACK)
screen.blit(label, (40,10))
pygame.display.update()

while not game_over:

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        if free_cells ==0:
            label = myfont.render("--DRAW--", 1, RED)
            screen.blit(label, (40,10))
            game_over = True

        if event.type == pygame.MOUSEMOTION:
            pygame.draw.rect(screen, WHITE, (0,0, width, SQUARESIZE))
            posx = event.pos[0]
            if turn == PLAYER:
                pygame.draw.circle(screen, R, (posx, int(SQUARESIZE/2)), RADIUS)

        pygame.display.update()

        if event.type == pygame.MOUSEBUTTONDOWN:
            if free_cells ==42:
                posx = event.pos[0]
                col = int(math.floor(posx/SQUARESIZE))
                if col==3:
                    label = notification.render("ILLEGAL MOVE! Choose another one", 1, RED)
                    screen.blit(label, (40,10))
                    continue
                else:
                    pygame.draw.rect(screen, WHITE, (0,0, width, SQUARESIZE))
            #print(event.pos)
            # Ask for Player 1 Input
            if turn == PLAYER:
                posx = event.pos[0]
                col = int(math.floor(posx/SQUARESIZE))

                if is_valid_location(board, col):
                    row = get_next_open_row(board, col)
                    drop_piece(board, row, col, PLAYER_PIECE)
                    free_cells -=1

                    if winning_move(board, PLAYER_PIECE):
                        label = myfont.render("YOU win!!", 1, RED)
                        screen.blit(label, (40,10))
                        game_over = True

                    turn += 1
                    turn = turn % 2

                    print_board(board)
                    draw_board(board)


    # # Ask for Player 2 Input
    # AI가 수를 두는 부분 - 김나연
    # 수를 계산하는 부분은 직접 짬
    if turn == AI and not game_over:     
        ai_row, ai_col = check_three(board, AI_PIECE)
        p_row, p_col = check_three(board, PLAYER_PIECE)
        
        # rule 첫번째 : 이기는 수가 있으면 무조건 거기에 두고 판을 끝낸다.
        if ai_row > -1 :
            drop_piece(board, ai_row, ai_col, AI_PIECE)
            free_cells -=1
            print("착수점 : ", ai_col+1)
            print("이유 : (", ai_row+1,",", ai_col+1,")에 두면 이기기 때문에")
            
        # rule 두번째 : 이기는 수가 없고, 다음판에 막을 수 있는 지는 수가 있으면 무조건 거기에 둔다.
        # 여러개가 있으면 어떡하지? -> 지는거지 뭐
        elif p_row> -1 :
            drop_piece(board, p_row, p_col, AI_PIECE)
            free_cells -=1
            print("착수점 : ", p_col+1)
            print("이유 : (", p_row+1,",", p_col+1,")에 상대방이 두면 이기기 때문에 ai가 여기 둬야한다")

        #세번째 : 앞의 두 경우가 아니면 나에게 가장 유리한 수를 둔다.
        else :
            line_col = [0,0,0,0,0,0,0]
            valid_locations = get_valid_locations(board)
            for col in valid_locations:
                row = get_next_open_row(board, col)
                line_col[col] = line_evaluation(board, AI_PIECE,row,col)
            value, second_value = minimax(board, 6, -math.inf, math.inf, True, line_col, -1)
            print("착수점 : ",value[0]+1)
            #[None, score + line,aithreat, plthreat, aidoublethreat, pldoublethreat, oddevenmixed, line]
            if value[2] == 100000000000000 :
                print("이유 : search 결과 이기는 수가 나왔기 때문에 ", value[0]+1,"을 선택하였다.")
            #best = [aithreat, plthreat, aidoublethreat, pldoublethreat, oddevenmixed, col,value,line_col[firstcol], firstcol]
            elif value[2] == -100000000000000 :
                print("이유 : search 결과 질 수 밖에 없다. ", value[0]+1,"을 선택하였다.")
            else :
                #print("마지막 프린트시에",second_value[7])
                print("이유 : search 결과 ", value[0]+1,"이라는 수를 둘 경우 \n ai의 threat이 ", value[2], \
                     "개, player의 threat이 ",value[3],"개, ai의 doublethreat이 ",value[4],\
                     "개, player의 doublethreat이 ",value[5],"개, oddevenmixed함수에 의한 값이 ",value[6],\
                     ", 생성 가능한 line 개수에 의한 값이", value[7],\
                     ", 총",value[1],"의 heuristic 값을 가지게 되어 가장 높은 heuristic값을 가지게 되므로 ",\
                     value[0]+1, "에 수를 두는 것이 최선이라고 판단된다.")
                if second_value[2] == 100000000000000 :
                      print("다른 수를 두지 않은 이유 : search 결과 이기는 수가 여러개 나왔고 그 중", value[0]+1,"을 선택하였다.")
                elif second_value[2] == -100000000000000 :
                      print("다른 수를 두지 않은 이유 : search 결과 지는 수여서 ", second_value[0]+1,"을 선택하지않았다.")
                else :
                      print(second_value[0]+1,"이라는 수를 둘 경우 \n ai의 threat이 ", second_value[2], \
                     "개, player의 threat이 ",second_value[3],"개, ai의 doublethreat이 ",second_value[4],\
                     "개, player의 doublethreat이 ",second_value[5],"개, oddevenmixed함수에 의한 값이 ",second_value[6],\
                     ", 생성 가능한 line 개수에 의한 값이", second_value[7],\
                     ", 총",second_value[1],"의 heuristic 값을 가지게 되어 ", value[0]+1,"이라는 수를 둘 때 보다 낮은 heuristic값을 가지게 되므로 ",\
                     second_value[0]+1, "에 수를 두는 것이 최선이 아니라고 판단된다.")        
        
            if is_valid_location(board, value[0]):
                row = get_next_open_row(board, value[0])
                drop_piece(board, row, value[0], AI_PIECE)
                free_cells -=1
 
                
        #이겼는지 여부 체크              
        if winning_move(board, AI_PIECE):
            label = myfont.render("AI win!!", 1, BLACK)
            screen.blit(label, (40,10))
            game_over = True

        print_board(board)
        draw_board(board)

        turn += 1
        turn = turn % 2
            
            
    if game_over:
        pygame.time.wait(5000)
        pygame.quit()



[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
착수점 :  5
이유 : search 결과  5 이라는 수를 둘 경우 
 ai의 threat이  0 개, player의 threat이  0 개, ai의 doublethreat이  0 개, player의 doublethreat이  0 개, oddevenmixed함수에 의한 값이  0 , 생성 가능한 line 개수에 의한 값이 5 , 총 5 의 heuristic 값을 가지게 되어 가장 높은 heuristic값을 가지게 되므로  5 에 수를 두는 것이 최선이라고 판단된다.
1 이라는 수를 둘 경우 
 ai의 threat이  0 개, player의 threat이  0 개, ai의 doublethreat이  0 개, player의 doublethreat이  0 개, oddevenmixed함수에 의한 값이  0 , 생성 가능한 line 개수에 의한 값이 0 , 총 -inf 의 heuristic 값을 가지게 되어  5 이라는 수를 둘 때 보다 낮은 heuristic값을 가지게 되므로  1 에 수를 두는 것이 최선이 아니라고 판단된다.
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 2. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 2. 0. 0.]]
착수점 :  4
이유 : search 결과  4 이라는 수를 둘 경우 
 a